# Global Forecast System Proof of Concept

This notebook is a proof of concept for working with the Global Forecast System (GFS). The GFS is a weather forecast model that publishes its data publicly to AWS S3. This notebook demonstrates how to access this data and plot it using Python.

In [ ]:
import os
from logging import debug, info, warning, error, critical

import logging

logging.basicConfig(
    level=logging.INFO,
    stream=os.sys.stdout,
)

## Data Download

The first step is to download the data. The data is stored in the `noaa-gfs-bdp-pds` bucket on AWS S3. The data is organized by year, month, day, and hour.

- Available GFS Parameter Sets: https://www.nco.ncep.noaa.gov/pmb/products/gfs/
- AWS S3 Bucket: https://noaa-gfs-bdp-pds.s3.amazonaws.com/index.html

The data is stored in GRIB2 format.

In [ ]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

import time
from datetime import timedelta
from concurrent.futures import ThreadPoolExecutor

from typing import List, Literal


def downloadGfsForecast(
    saveDir: str,
    forecastHours: int = 24,
    gridResolution: Literal["1p00", "0p50", "0p25"] = "1p00",
    tNow: time.struct_time = time.gmtime(),
) -> List[str]:
    GFS_BUCKET_NAME = "noaa-gfs-bdp-pds"
    GFS_BUCKET_REGION = "us-east-1"

    s3 = boto3.resource(
        "s3",
        region_name=GFS_BUCKET_REGION,
        config=Config(
            signature_version=UNSIGNED,  # Unsigned as the bucket is public
            max_pool_connections=100,  # Else get a warning
        ),
    )

    yyymmdd = time.strftime("%Y%m%d")
    lastHourlyQuarter = int(time.strftime("%H", tNow)) // 6 * 6
    hh = f"{lastHourlyQuarter:02d}"

    bucketSubdir = f"gfs.{yyymmdd}/{hh}/atmos"
    fileNames = [
        f"gfs.t{hh}z.pgrb2.{gridResolution}.f{i:03d}"  # Assume pgrb2 as common parameter set
        for i in range(0, forecastHours + 1)
    ]

    if gridResolution == "0p50" or gridResolution == "1p00":
        debug(f"{gridResolution} implies 3-hourly forecast intervals")
        fileNames = fileNames[::3]

    os.makedirs(saveDir, exist_ok=True)

    existingFiles = os.listdir(saveDir)

    filesToDownload = [file for file in fileNames if file not in existingFiles]

    def downloadFromS3(filename, saveDir=saveDir):
        """
        Download process for a single GFS file from AWS S3
        """
        info(f"Downloading {filename}")

        try:
            s3.Bucket(GFS_BUCKET_NAME).download_file(
                f"{bucketSubdir}/{filename}",
                f"{saveDir}/{filename}",
            )
        except Exception as e:
            error(f"Error downloading {filename}: {e}")

    with ThreadPoolExecutor() as executor:
        executor.map(downloadFromS3, filesToDownload)

    existingFiles = os.listdir(saveDir)  # Update list of existing files
    for file in existingFiles:
        if file not in fileNames:
            warning(
                f"File {file} in {saveDir} is not part of the current forecast. Deleting."
            )
            os.remove(f"{saveDir}/{file}")

    return os.listdir(saveDir)


dataSaveDir = f"Data/{time.strftime('%Y%m%d', time.gmtime())}"
forecastHours = 24 * 2
forecastFiles = downloadGfsForecast(
    saveDir=dataSaveDir,
    gridResolution="0p50",
    forecastHours=forecastHours,
    # Set time to be the current GMT time minus 6 hours to ensure that the latest forecast is downloaded
    tNow=time.gmtime(time.time() - 6 * 3600),
)

## Data Exploration

The `cfgrib` library is used to read the GRIB2 files. 

Example data is explored to understand the structure of the data.

In [ ]:
import xarray as xr

ds = xr.open_dataset(
    os.path.join(dataSaveDir, forecastFiles[0]),
    engine="cfgrib",
    filter_by_keys={
        "typeOfLevel": "surface",
        "stepType": "instant",
    },
)

In [ ]:
for v in ds:
    print(f"{v}\t{ds[v].attrs['long_name']}, {ds[v].attrs['units']}")

In [ ]:
print(ds)

In [ ]:
temperature = ds.get("t")

print(temperature)

## Plotting

`matplotlib` can plot the data with the appropriate projections.

In [ ]:
latitudes = temperature.latitude.values
longitudes = temperature.longitude.values

import matplotlib.pyplot as plt
import cartopy.crs as ccrs

fig = plt.figure(figsize=(10, 5))
ax = fig.add_subplot(1, 1, 1, projection=ccrs.SouthPolarStereo())

t_celsius = temperature - 273.15
t_celsius.plot(
    ax=ax,
    transform=ccrs.PlateCarree(),
    cmap="coolwarm",
    cbar_kwargs={"shrink": 0.4},
)

ax.coastlines()
ax.set_extent([-180, 180, -90, -60], ccrs.PlateCarree())
ax.gridlines(
    crs=ccrs.PlateCarree(),
    draw_labels=True,
    linewidth=1,
    color="gray",
    alpha=0.5,
    linestyle="--",
)

SANAE_IV = (-71.673611, -2.828611)
ax.plot(
    SANAE_IV[1],
    SANAE_IV[0],
    "ro",
    transform=ccrs.PlateCarree(),
    label="SANAE IV",
)

## Local Forecast

The GFS data local to a specific location can be extracted and plotted.

In [ ]:
# Extract the temperature at SANAE IV and save as timeseries for all forecast files

import pandas as pd

temperatureForecast = []

for file in forecastFiles:
    ds = xr.open_dataset(
        os.path.join(dataSaveDir, file),
        engine="cfgrib",
        filter_by_keys={
            "typeOfLevel": "surface",
            "stepType": "instant",
        },
    )

    temperature = ds.get("t")
    temperatureSanae = temperature.sel(
        latitude=SANAE_IV[0], longitude=SANAE_IV[1], method="nearest"
    )
    timeSanae = ds.valid_time.values

    temperatureForecast.append((timeSanae, temperatureSanae.values - 273.15))

temperatureForecast.sort(key=lambda x: x[0])
temperatureForecast = pd.DataFrame(temperatureForecast, columns=["Time", "Temperature"])

In [ ]:
with plt.style.context("dark_background"):
    temperatureForecast.plot(
        x="Time",
        y="Temperature",
        marker="o",
        title="Temperature forecast at SANAE IV",
        ylabel="Temperature [°C]",
        xlim=(
            temperatureForecast.Time.min(),
            temperatureForecast.Time.min() + pd.Timedelta(hours=forecastHours),
        ),
        figsize=(16, 9),
        grid=True,
        legend=False,
    )